## Assignment: Segmentation and clustering of neighborhood in Toronto

## 1. Fetch the data from wikipedia by scraping 

In [6]:
!pip install folium

     |████████████████████████████████| 93 kB 4.1 MB/s  eta 0:00:01


In [9]:
# First we will import essential libraries
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import folium

In [18]:
main_source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(main_source, 'lxml')

Table = soup.find("table")
Table_row = Table.tbody.find_all('tr')

res = []
for tr in Table_row:
    td = tr.find_all("td")
    row = [tr.text for tr in td]

    if row != [] and row != "Not assigned":
        
        if "Not assigned" in row[2]:
            row[2] = row[0]
        res.append(row)

web_df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
web_df = web_df.replace('\n','',regex=True)
web_df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [27]:
web_df = web_df[web_df.Borough != 'Not assigned']
web_df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [32]:
web_df1 = web_df.reset_index(drop = True)
web_df1

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [33]:
print("shape: ", web_df1.shape)

shape:  (103, 3)


***

## 2. Since over data frame is ready we will merge it with latitude and longitude columns, which we will fetch from geospatial csv file

In [35]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**we have loaded the data, so now we will merge the coordinates data with the data consisting the Borough adn Neighborhood**

In [36]:
toronto_df = pd.merge(web_df1, df_data_1, how='left', left_on = 'PostalCode',
                     right_on = 'Postal Code')
toronto_df.drop("Postal Code", axis=1, inplace=True)
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


**We have the data frame with the location of the neighborhood and we can now perform clustering with the above data frame**